In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os as os

DƯ LIỆU VỀ Diện tích, dân số và mật độ dân số phân theo địa phương

In [2]:
raw_data = pd.read_excel('Datasets/Crawl_data/data.xlsx')
def formatting(i): # dong muon format
    row = raw_data.iloc[i] # du lieu tai dong do 2011 -> 2024
    year_data = [] # co du lieu tai cac nam do
    cnt = 0
    for i in range(1,42,3): # 1 -> 42
        if 2011 + cnt < 2025:
            a = [row[f'{2011 + cnt}']] # a = [row['2011']] 
        else: break
        cnt += 1
        for j in range(1,3):
            a.append(row[f'Unnamed: {i+j}']) # a = []
        year_data.append(a) # year = [data 2011] index = 0
    region = row['Unnamed: 0'] # string -> list
    years = range(2011,2011+len(year_data)) #
    index = pd.MultiIndex.from_product(
        [[region],years],names = ['Vùng','Năm'] # dat ten columns
    )
    return pd.DataFrame(year_data,index=index,columns = list('abc'))
Raw_data = formatting(1)
for i in range(2,71):
    Raw_data = pd.concat([Raw_data,formatting(i)])
Raw_data = Raw_data.rename(columns={'a': 'Diện tích(Km2)','b':'Dân số trung bình (Nghìn người)','c':'Mật độ dân số (Người/km2)'})
Raw_data.to_excel('Datasets/Raw_data/Area_population_density.xlsx')

Dữ liệu về Dân số trung bình phân theo địa phương, giới tính và thành thị nông thôn

In [3]:
raw_data = pd.read_excel('Datasets/Crawl_data/Data_02.xlsx',engine='openpyxl')
def get_data(str1,str2,row):
    if str2 == 'end':
        return np.array([data for data in row[str1:]])
    cols = row.index
    start = cols.get_loc(str1)
    end = cols.get_loc(str2)
    return row.iloc[start:end].to_numpy()
def formatting_2(i):
    row = raw_data.iloc[i]
    
    sum_data = get_data('Tổng số','Nam',row)
    man_data = get_data('Nam','Nữ',row)
    woman_data = get_data('Nữ','Thành thị',row)
    city_data = get_data('Thành thị','Nông thôn',row)
    rural_data = get_data('Nông thôn','end',row)

    year_data = np.column_stack((sum_data,man_data,woman_data,city_data,rural_data))
    
    year = [i for i in range(2011,2025)]
    region = row['Vùng']
    
    index = pd.MultiIndex.from_product([[region],year],names=['Vùng','Năm'])
    return pd.DataFrame(year_data,index=index,columns=list('abcde'))

Raw_data_2 = formatting_2(1)
for i in range(2,71):
    Raw_data_2 = pd.concat([Raw_data_2,formatting_2(i)])
Raw_data_2 = Raw_data_2.fillna(method='ffill')
Raw_data_2 = Raw_data_2.rename(columns={'a': 'Tổng số', 'b': 'Nam', 'c': 'Nữ','d':'Tổng dân số thành thị','e':'Tổng dân số nông thôn'})
Raw_data_2.to_excel('Datasets/Raw_data/Average_population_by_sex_urban_rural_residence.xlsx')

C:\Users\Admin\AppData\Local\Temp\ipykernel_2100\555582010.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  Raw_data_2 = Raw_data_2.fillna(method='ffill')
C:\Users\Admin\AppData\Local\Temp\ipykernel_2100\555582010.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Raw_data_2 = Raw_data_2.fillna(method='ffill')


Data_05

In [4]:
raw_data_05 = pd.read_excel('Datasets/Crawl_data/Data_05.xlsx')

def formatting_data_05(i):
    """Format one region's data row into long format like 'Sex_ratio_locality_true'"""
    row = raw_data_05.loc[i]
    region = row['Unnamed: 0']

    # --- Collect all yearly data ---
    year_data = []
    for col in row.index[1:]:
        # Extract only valid numeric year columns
        year_str = str(col)
        year = ''.join(filter(str.isdigit, year_str))
        if year.isdigit():
            value = row[col]
            if pd.notna(value) and value != '..':  # Skip missing or placeholder
                year_data.append([region, int(year), float(value)])

    # Convert to DataFrame for that region
    return pd.DataFrame(year_data, columns=['Vùng', 'Năm', 'Tỷ lệ nam nữ'])


# === COMBINE ALL REGIONS ===
formatted_data_05 = formatting_data_05(0)
for i in range(1, len(raw_data_05)):
    formatted_data_05 = pd.concat([formatted_data_05, formatting_data_05(i)], ignore_index=True)

# === OUTPUT ===
formatted_data_05.reset_index(drop=True, inplace=True)
formatted_data_05.to_excel('Datasets/Raw_data/Sex_ratio_locality.xlsx', index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2100\1795997561.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  formatted_data_05 = pd.concat([formatted_data_05, formatting_data_05(i)], ignore_index=True)


Data_08

In [5]:
raw_data_08 = pd.read_excel('Datasets/Crawl_data/Data_08.xlsx')
def formatting_data_08(i):
    row = raw_data_08.loc[i]
    year_data = []
    Crude_birth_rate = get_data('Tỷ suất sinh thô','Tỷ suất chết thô',row)
    Crude_death_rate = get_data('Tỷ suất chết thô','Tỷ lệ tăng tự nhiên',row)
    Natural_birht_rate = get_data('Tỷ lệ tăng tự nhiên','end',row)
    
    year_data = np.column_stack([Crude_birth_rate,Crude_death_rate,Natural_birht_rate])
    year = [i for i in range(2011,2025)]
    region = row['Unnamed: 0']
    
    index = pd.MultiIndex.from_product([[region],year],names=['Vùng','Năm'])
    return pd.DataFrame(year_data,index=index,columns=list(['Tỷ suất sinh thô','Tỷ suất chết thô','Tỷ lệ tăng tự nhiên']))
data_08 = formatting_data_08(1)
for i in range(2,71):
    data_08 = pd.concat([data_08,formatting_data_08(i)])
data_08.to_excel('Datasets/Raw_data/Crude_birth_death_rate_natural_growth_rate_locality.xlsx')
    

Data 15

In [6]:
raw_data_15 = pd.read_excel('Datasets/Crawl_data/Data_15.xlsx')
def get_data(str1,str2,row):
    if str2 == 'end':
        return np.array([data for data in row[str1:]])
    cols = row.index
    start = cols.get_loc(str1)
    end = cols.get_loc(str2)
    return row.iloc[start:end].to_numpy()
def formatting_data_15(i):
    row = raw_data_15.loc[i]
    year_data = []
    a = get_data('Tỷ suất nhập cư','Tỷ suất xuất cư',row)
    b = get_data('Tỷ suất xuất cư','Tỷ suất di cư thuần',row)
    c = get_data('Tỷ suất di cư thuần','end',row)
   
    year_data = np.column_stack([a,b,c])
    year = [i for i in range(2011,2025)]
    region = row['Unnamed: 0']
    
    index = pd.MultiIndex.from_product([[region],year],names=['Vùng','Năm'])
    return pd.DataFrame(year_data,index=index,columns=list(['Tỷ suất nhập cư','Tỷ suất nhập cư','Tỷ suất di cư thuần']))
formatting_data_15(1)
data_15 = formatting_data_15(1)
for i in range(2,71):
    data_15 = pd.concat([data_15,formatting_data_15(i)])
data_15.to_excel('Datasets/Raw_data/Immigration_emigration_net_migration_rates_locality.xlsx')


Data 25



In [ ]:
raw_data_25 = pd.read_excel('Datasets/Crawl_data/Data_25_test.xlsx')

In [8]:
raw_data_25.to_excel('Datasets/Raw_data/Unemployed_Data.xlsx',index=False)